# Search stocks

In [1]:
import sys
!{sys.executable} -m pip install pandas_datareader yfinance matplotlib

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
import pandas as pd
from pandas import DataFrame
import json
from pandas_datareader import data as pdr
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt



In [3]:
yf.pdr_override()
#  https://datahub.io/core/nasdaq-listings
df = pd.read_csv('https://pkgstore.datahub.io/core/nasdaq-listings/nasdaq-listed_csv/data/7665719fb51081ba0bd834fde71ce822/nasdaq-listed_csv.csv')
# convert symbol column to list

tickers = df['Symbol'].values.tolist()
#t = [x.replace('.', '-') for x in tickers] 
#print(tickers)
tickers
#yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30")

['AAIT',
 'AAL',
 'AAME',
 'AAOI',
 'AAON',
 'AAPL',
 'AAVL',
 'AAWW',
 'AAXJ',
 'ABAC',
 'ABAX',
 'ABCB',
 'ABCD',
 'ABCO',
 'ABCW',
 'ABDC',
 'ABGB',
 'ABIO',
 'ABMD',
 'ABTL',
 'ABY',
 'ACAD',
 'ACAS',
 'ACAT',
 'ACET',
 'ACFC',
 'ACFN',
 'ACGL',
 'ACHC',
 'ACHN',
 'ACIW',
 'ACLS',
 'ACNB',
 'ACOR',
 'ACPW',
 'ACRX',
 'ACSF',
 'ACST',
 'ACTA',
 'ACTG',
 'ACTS',
 'ACUR',
 'ACWI',
 'ACWX',
 'ACXM',
 'ADAT',
 'ADBE',
 'ADEP',
 'ADES',
 'ADHD',
 'ADI',
 'ADMA',
 'ADMP',
 'ADMS',
 'ADNC',
 'ADP',
 'ADRA',
 'ADRD',
 'ADRE',
 'ADRU',
 'ADSK',
 'ADTN',
 'ADUS',
 'ADVS',
 'ADXS',
 'ADXSW',
 'AEGN',
 'AEGR',
 'AEHR',
 'AEIS',
 'AEPI',
 'AERI',
 'AETI',
 'AEY',
 'AEZS',
 'AFAM',
 'AFCB',
 'AFFX',
 'AFH',
 'AFMD',
 'AFOP',
 'AFSI',
 'AGEN',
 'AGII',
 'AGIIL',
 'AGIO',
 'AGNC',
 'AGNCB',
 'AGNCP',
 'AGND',
 'AGRX',
 'AGTC',
 'AGYS',
 'AGZD',
 'AHGP',
 'AHPI',
 'AIMC',
 'AINV',
 'AIQ',
 'AIRM',
 'AIRR',
 'AIRT',
 'AIXG',
 'AKAM',
 'AKAO',
 'AKBA',
 'AKER',
 'AKRX',
 'ALCO',
 'ALDR',
 'ALDX',
 'AL

In [4]:
ma_count = 21

print(dt.datetime(2021,1,1))
data = pdr.get_data_yahoo(tickers, start=dt.datetime(2021,1,1), end=dt.date.today())['Close']
data.tail()


2021-01-01 00:00:00
[*********************100%***********************]  2967 of 2967 completed

1306 Failed downloads:
- ARRS: No data found, symbol may be delisted
- OGXI: No data found for this date range, symbol may be delisted
- ANCX: No data found for this date range, symbol may be delisted
- XLNX: No data found, symbol may be delisted
- FINL: No data found for this date range, symbol may be delisted
- PBMD: No data found for this date range, symbol may be delisted
- NFEC: No data found for this date range, symbol may be delisted
- TRTLU: No data found, symbol may be delisted
- BLVDU: No data found for this date range, symbol may be delisted
- ZIOP: No data found, symbol may be delisted
- LAYN: No data found for this date range, symbol may be delisted
- SSFN: No data found, symbol may be delisted
- UBNT: No data found, symbol may be delisted
- ADAT: No data found for this date range, symbol may be delisted
- SHPG: No data found for this date range, symbol may be delisted
- AXPW: N

,AAIT,AAL,AAME,AAOI,AAON,AAPL,AAVL,AAWW,AAXJ,ABAC,...,ZIONZ,ZIOP,ZIV,ZIXI,ZLTQ,ZN,ZNGA,ZSPH,ZU,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
2022-07-06 00:00:00,NaN,13.89,2.73,1.53,55.939999,142.919998,NaN,59.049999,67.860001,NaN,...,NaN,NaN,28.9,NaN,NaN,NaN,NaN,NaN,NaN,25.010000
2022-07-07 00:00:00,NaN,14.10,3.03,1.62,55.779999,146.350006,NaN,60.919998,69.279999,NaN,...,NaN,NaN,28.9,NaN,NaN,NaN,NaN,NaN,NaN,25.799999
2022-07-08 00:00:00,NaN,13.97,3.06,1.63,54.720001,147.039993,NaN,61.669998,69.209999,NaN,...,NaN,NaN,28.9,NaN,NaN,NaN,NaN,NaN,NaN,25.680000
2022-07-11 00:00:00,NaN,13.43,3.13,1.50,54.419998,144.869995,NaN,60.490002,67.239998,NaN,...,NaN,NaN,28.9,NaN,NaN,NaN,NaN,NaN,NaN,25.309999
2022-07-12 00:00:00,NaN,14.77,3.11,1.52,55.119999,145.860001,NaN,60.419998,67.050003,NaN,...,NaN,NaN,28.9,NaN,NaN,NaN,NaN,NaN,NaN,25.360001


In [5]:
ma = data.rolling(window=ma_count).mean()
ma_pers = data.div(ma).mul(100).sub(100)
#ma_pers.iloc[[-1]]
ma_pers_r=ma_pers.iloc[::-1]
d = {}
df_prep = []
for col_name in ma_pers_r.columns:
    d[col_name]=0
    ma_last = None
    for index, row in ma_pers_r.iterrows():
        if row[col_name] > 0:
            d[col_name] += 1
            if ma_last == None:
                ma_last=row[col_name]
        else:
            if d[col_name] != 0:
               df_prep.append([col_name, d[col_name], ma_last, ma_count, "daily"]) 
            break
df = DataFrame(df_prep,columns=['name','count', 'ma_diff', 'ma_size', 'ma_type']).sort_values(by=['ma_diff', 'count'], ascending=False)
print(df['count'][df['count'] >= 10].size)
lead_comps_df = df[df['count'] >= 10]
lead_comps = lead_comps_df['name'].values.tolist()
lead_comps_df

0


,name,count,ma_diff,ma_size,ma_type


In [6]:
for quote in lead_comps:
    fig, ax = plt.subplots(figsize=(20, 10))
    plt.plot(data[quote], label=quote); 
    plt.plot(ma[quote], label = 'MA');
    plt.legend(loc=2);